<a href="https://colab.research.google.com/github/Colley-K/Time_series_forecasting/blob/master/core_items_cateogries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# State of the Union

#### Where are we at?

* First, we tried to model the data using categories from 2.5 years ago of historic data. It was then that we learned the data was random walk processes and could not be modelled when looking out this far. I did about 2 dozen different combinations of tests looking for signal in the data. Mixing and matching different forecast windows, historic data, and converted data. We finally found some winners! It looks like 1.5 or 1 year of history with daily forecast windows were the most promising indication of signal.

> * The reason it is random is because 2018 represents an unlikely trend when they lost business from one of their major customers. During this first attempt, we revealed that going back one year from todays date will yeild the most accurate historic trends. **This will definitely be something to watch in the future when their business experiences different ups and downs from outside influences**

* Next, with the one year + daily forecasts in mind, we went back and cleaned up the categories to reflect the pre-selected categories the customer created. The plan was to create a hierarchical model for each category that would then be dispersed to the individual PLU level. What happened? The individual PLU level represented random walk processes again, and the hierarchical models were not beating a "naive forecast" (aka using yesterdays value to predict todays value). Think of a heart monitor display... That's what most of the items looked like on the individual level. Baselining then random spikes in demand. 

* Clearly this is an issue with the scale of data we are dealing with. This small grocery distributor did not have the luxury of large datasets, and a lot of the products may only be ordered a handful of times. By definition, the forecast can only be as good as the data, and thus we had to figure out a work around for this. I wasn't ready to give up so easily! 

* Now here we are, this notebook represents an attempt to model the top 100 products (by revenue) that were ordered last year.

# Purpose of this Notebook

#### *The goal of this notebook is to establish a proof of concept. Is it possible to create a model for the 100 most ordered items? If we cannot beat the Naive forecast, then the data will not allow for an accurate forecast and I will reach the end of the road (well ok I will still try a regression model taking out the time series component). Regardless, this will be a pivotal test of what is possible given the data.*

* This notebook in particular will focus of extracting the 100 most ordered items in the past year


In [0]:
#Colab stuffs:

#get a fast operator system
!nvidia-smi

#mount google drive
from google.colab import drive
drive.mount('/content/drive')#click on the link it provides and copy and paste that code into the authorization area

#access the OS system to work with current directories:
import os 

Wed Nov  6 14:51:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
plt.style.use('fivethirtyeight')
sns.set_style("whitegrid")
sns.despine()
sns.set(rc={'figure.figsize':(15,9)})
%matplotlib inline

In [0]:
df = pd.read_csv('drive/My Drive/Capstone_2/data/one_year.csv', low_memory=False)
dfn = pd.read_csv('drive/My Drive/Capstone_2/data/new_info.csv', low_memory=False)
dfm = pd.read_csv('drive/My Drive/Capstone_2/data/master.csv', low_memory=False)

#artran = pd.read_csv('drive/My Drive/Capstone_2/data/artran.csv', low_memory=False) #original DF
#df = pd.read_csv('drive/My Drive/Capstone_2/data/two_year.csv', low_memory=False)
#uom = pd.read_excel('drive/My Drive/Capstone_2/data/ItemUOM.xlsx') #unit of measure spreadsheet

In [0]:
df.shape

(209225, 31)

In [0]:
#Counting Null values in df
print (df.isna().sum())
print (dfn.isna().sum())
print(dfm.isna().sum())

#getting rid of irrelevant columns
#df.drop(columns= ['route', 'salesman'], inplace= True)
dfn.drop(columns= ['route', 'salesman'], inplace= True)
dfm.drop(columns= ['route', 'salesman'], inplace= True)

invdate      0
invno        0
item         0
desc         0
uom          0
units        0
price        0
totalcost    0
ordqty       0
shipqty      0
amount       0
custno       0
sono_x       0
sono_y       0
paid         0
duedate      0
days         0
status       0
datesold     0
delivery     0
priccode     0
orgprice     0
firstrun     0
added        0
invline      0
dtype: int64
invdate        0
invno          0
item           0
desc           0
uom            0
units          0
price          0
totalcost      0
ordqty         0
shipqty        0
amount         0
custno         0
sono_x         0
sono_y         0
paid           0
duedate        0
days           0
status         0
datesold       0
delivery       0
route        549
salesman     293
priccode       0
orgprice       0
firstrun       0
added          0
invline        0
dtype: int64
invdate         0
invno           0
item            0
desc            0
uom             0
units           0
price           0
totalcost     

In [0]:
#Dropping the negative values because they represent returns and we are only measuring demand/ returns is a different problem

#counting negative values
print (f'Number of Negative Order Quantities Values: {np.sum((df.ordqty < 0).values.ravel())}') 
print (f'Number of credits given to customer: {np.sum((df.price < 0).values.ravel())}') 


#dropping the return or negative value rows

def negative_nums(df):
  df[df.ordqty < 0] = np.nan
  df[df.price < 0] = np.nan
  df.dropna(inplace=True)

negative_nums(df)
negative_nums(dfn)
negative_nums(dfm)

Number of Negative Order Quantities Values: 7699
Number of credits given to customer: 1074


In [0]:
def create_cols (df):
  #Creating a normalized Order Qty column
  df['norm_qty'] = df['ordqty'] / df['units']

  #Creating a revenue column per item per day
  df['revenue'] = df['shipqty'] * df['price']

  #creating a column for the total orders per item
  df['total_qty'] = df.groupby('item')["norm_qty"].transform('sum')

  #creating a column for the total revenue per item
  df['total_rev'] = df.groupby('item')["revenue"].transform('sum')

  #splitting off the first phrase from the item's description, and turning them into lower case
  df['label'] = df['desc'].str.split(",").str[0].str.lower()
  df['label'] = df['label'].str.split(" ").str[0].str.lower()


In [0]:
create_cols(df)
create_cols(dfn)
create_cols(dfm)

In [0]:
dfm.tail()

,invdate,invno,item,desc,uom,units,price,totalcost,ordqty,shipqty,amount,custno,sono_x,sono_y,paid,duedate,days,status,datesold,delivery,priccode,orgprice,firstrun,added,invline,norm_qty,revenue,total_qty,total_rev,label
242494,2019-10-24,284834.0,20318B,"Root, Ginger, Yellow Industrial 30lb, Cert. Org.",1LB,30.0,4.9,1.20,1.0,1.0,345.0,30-PMOTH,293553.0,293553.0,0.0,2019-10-31,7.0,O,2019-10-24,2019-10-24,PL6,4.9,1.0,2019-10-23 17:26:25,55.0,0.033333,4.9,1481.100000,103190.82,root
242495,2019-10-24,284834.0,20306B,"Garlic, 22lb Certified Organic",5LB,4.4,22.9,14.09,1.0,1.0,345.0,30-PMOTH,293553.0,293553.0,0.0,2019-10-31,7.0,O,2019-10-24,2019-10-24,PL6,22.9,1.0,2019-10-23 17:26:25,82.0,0.227273,22.9,34.318182,2869.70,garlic
242496,2019-10-24,284834.0,10401,"Grapes, Black Seedless, Bagged, 19lb Cert. OG",HALF,2.0,21.9,12.50,1.0,1.0,345.0,30-PMOTH,293553.0,293553.0,0.0,2019-10-31,7.0,O,2019-10-24,2019-10-24,PL6,21.9,1.0,2019-10-23 17:26:25,83.0,0.500000,21.9,90.000000,3069.20,grapes
242497,2019-10-24,284834.0,60007,"Mushroom, Button Medium 5lb, Certified Organic",CASE,1.0,19.9,11.50,1.0,0.0,345.0,30-PMOTH,293553.0,293553.0,0.0,2019-10-31,7.0,O,2019-10-24,2019-10-24,PL6,19.9,1.0,2019-10-23 17:26:25,85.0,1.000000,0.0,645.400000,10638.06,mushroom
242498,2019-10-24,284834.0,CV30378,"Onion, Red Pearl CV",CASE,1.0,45.9,30.00,1.0,1.0,345.0,30-PMOTH,293553.0,293553.0,0.0,2019-10-31,7.0,O,2019-10-24,2019-10-24,PL6,45.9,1.0,2019-10-23 17:26:25,86.0,1.000000,45.9,6.000000,178.50,onion


In [0]:
df.columns

Index(['invdate', 'index', 'invno', 'item', 'desc', 'uom', 'units', 'price',
       'totalcost', 'ordqty', 'shipqty', 'diff_quantity', 'amount', 'profit',
       'custno', 'sono_x', 'sono_y', 'paid', 'diff_amt_paid', 'duedate',
       'days', 'status', 'datesold', 'delivery', 'priccode', 'orgprice',
       'firstrun', 'added', 'invline', 'norm_qty', 'revenue', 'total_qty',
       'total_rev', 'label'],
      dtype='object')

In [0]:
#creating a separate DF with the 100 top selling items by revenue
arr = np.sort(df['total_qty'].unique())[-100:]
print (f'100 Largest Revenue Items: \n {arr}')

df2= df[df['total_qty'].isin(arr)]

100 Largest Revenue Items: 
 [  792.           793.375        799.5          805.26
   807.           809.           826.75         834.90625
   841.           845.5          860.           874.
   892.8          912.26315789   929.           948.5
   951.8          960.25         962.5          973.5
   986.25         986.5         1003.9         1005.
  1007.          1017.5         1022.          1044.05
  1060.          1084.625       1091.          1093.32
  1117.          1124.          1147.75        1173.71
  1181.          1187.5         1189.          1220.75
  1226.7         1234.          1235.          1254.94
  1298.5         1304.3         1358.          1380.5
  1385.56666667  1389.5         1421.6         1435.
  1486.          1488.3         1493.5         1560.25
  1595.          1598.43333333  1653.          1653.33
  1660.          1689.          1698.04        1772.5
  1805.          1807.8         1820.25        1891.
  1917.85        1924.          1930.9       

In [0]:
#Cleaning up the columns:

df2 = df2[['invdate', 'item', 'desc', 'uom', 'units', 'price',
       'totalcost', 'ordqty', 'shipqty', 'norm_qty', 'revenue', 'total_qty',
       'total_rev', 'label']]

In [0]:
#Double checking there are only 100 items in the list!
df2.nunique()

invdate       363
item          101
desc          158
uom            12
units          12
price         338
totalcost     548
ordqty        133
shipqty       159
norm_qty      169
revenue      1713
total_qty     100
total_rev     101
label          45
dtype: int64

In [0]:
df2.to_csv(r'/content/drive/My Drive/Capstone_2/data/100Q_items.csv', index= 'invdate', header=True)
df2.to_csv(r'/content/drive/My Drive/Capstone_2/data/100Q_items.csv', index= 'invdate', header=True)
df2.to_csv(r'/content/drive/My Drive/Capstone_2/data/100Q_items.csv', index= 'invdate', header=True)

In [0]:
#create a pivot table of the categories

#df2 = pd.pivot_table(df, values= "ordqty", index= "invdate", columns = "cat", aggfunc=np.sum, fill_value=0)
